# Question Answering Challenge

### Using a Wikipedia dump for offline retrieval

As specified in https://en.wikipedia.org/wiki/Wikipedia:Database_download, we download the Polish Wikipedia dump from https://dumps.wikimedia.org/.

In [ ]:
import requests
from tqdm import tqdm

url = 'https://dumps.wikimedia.org/plwiki/latest/plwiki-latest-pages-articles.xml.bz2'

response = requests.get(url, stream=True)
if response.status_code == 200:
    file_name = 'plwiki-latest-pages-articles.xml.bz2'
    total_size = int(response.headers.get('content-length', 0))
    chunk_size = 1024  # 1 KB
    with open(file_name, 'wb') as file, tqdm(
        desc=file_name,
        total=total_size,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for chunk in response.iter_content(chunk_size=chunk_size):
            if chunk:
                file.write(chunk)
                bar.update(len(chunk))
    print(f'Wikipedia dump downloaded successfully: {file_name}')
else:
    print(f"Failed to download the Wikipedia dump. HTTP Status Code: {response.status_code}")


plwiki-latest-pages-articles.xml.bz2: 100%|██████████| 2.35G/2.35G [2:06:48<00:00, 332kB/s]    

Wikipedia dump downloaded successfully: plwiki-latest-pages-articles.xml.bz2


: 

### Extracting data from the Wikipedia dump using WikiExtractor

Run the following command to extract articles from the dump into an `extracted/` folder:
```bash
wikiextractor --json plwiki-latest-pages-articles.xml.bz2 -o extracted
```

### Processing the extracted data and indexing it into Elasticsearch

In [ ]:
import os
import json
import logging
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk

# -------------------------------------------
# KONFIGURACJA
# -------------------------------------------
INDEX_NAME = "wiki_index"

# Mappings (uwaga: "paragraph_number" to "integer", nie "number")
INDEX_BODY = {
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "paragraph_number": {"type": "integer"},
            "content": {"type": "text"}
        }
    }
}

# Adres Elasticsearch (zakładamy, że działa na http://localhost:9200)
es = Elasticsearch("http://localhost:9200")

# -------------------------------------------
# UTWORZENIE INDEKSU (jeśli nie istnieje)
# -------------------------------------------
if not es.indices.exists(index=INDEX_NAME):
    es.indices.create(index=INDEX_NAME, body=INDEX_BODY)
    # Wyłączamy odświeżanie i replikę na czas indeksowania (przyspieszy to import)
    es.indices.put_settings(
        index=INDEX_NAME,
        body={
            "index": {
                "refresh_interval": "-1",
                "number_of_replicas": 0
            }
        }
    )
    print(f"Created index '{INDEX_NAME}' with temporary settings (no refresh, 0 replicas).")

# -------------------------------------------
# DEFINICJA GENERATORA DOKUMENTÓW
# -------------------------------------------
def generate_actions(dump_dir, index_name):
    """
    Generator dla parallel_bulk. Dla każdego pliku wiki_... wyciąga artykuły,
    dzieli je na paragrafy i yielduje dokumenty do zindeksowania.
    """
    for root, _, files in os.walk(dump_dir):
        for file in files:
            if file.startswith('wiki_'):
                file_path = os.path.join(root, file)
                print(f"Processing file: {file_path}")

                with open(file_path, 'r', encoding='utf-8') as f:
                    for line in f:
                        article = json.loads(line)
                        title = article.get('title', '')
                        content = article.get('text', '')

                        paragraphs = content.split("\n\n")
                        for i, paragraph in enumerate(paragraphs):
                            paragraph = paragraph.strip()
                            if paragraph:
                                yield {
                                    "_index": index_name,
                                    "_source": {
                                        "title": title,
                                        "paragraph_number": i,
                                        "content": paragraph
                                    }
                                }

# -------------------------------------------
# WYWOŁANIE parallel_bulk DO MASOWEGO INDEKSOWANIA
# -------------------------------------------
actions = generate_actions("extracted", INDEX_NAME)

successes = 0
for ok, resp in parallel_bulk(
    client=es,
    actions=actions,
    thread_count=4,      # liczba wątków
    chunk_size=500,      # wielkość jednej paczki dokumentów
    max_chunk_bytes=10 * 1024 * 1024  # ~10 MB na paczkę
):
    if not ok:
        logging.error(f"Error indexing chunk: {resp}")
    else:
        successes += 1

print(f"Successfully processed {successes} chunks of data.")

# -------------------------------------------
# PRZYWRACANIE NORMALNYCH USTAWIEŃ
# -------------------------------------------
es.indices.put_settings(
    index=INDEX_NAME,
    body={
        "index": {
            "refresh_interval": "1s",
            "number_of_replicas": 1
        }
    }
)
print(f"Indexing complete. Restored normal settings for '{INDEX_NAME}'.")


Processing file: extracted\AA\wiki_00
Processing file: extracted\AA\wiki_01
Processing file: extracted\AA\wiki_02
Processing file: extracted\AA\wiki_03
Processing file: extracted\AA\wiki_04
Processing file: extracted\AA\wiki_05
Processing file: extracted\AA\wiki_06
Processing file: extracted\AA\wiki_07
Processing file: extracted\AA\wiki_08
Processing file: extracted\AA\wiki_09
Processing file: extracted\AA\wiki_10
Processing file: extracted\AA\wiki_11
Processing file: extracted\AA\wiki_12
Processing file: extracted\AA\wiki_13
Processing file: extracted\AA\wiki_14
Processing file: extracted\AA\wiki_15
Processing file: extracted\AA\wiki_16
Processing file: extracted\AA\wiki_17
Processing file: extracted\AA\wiki_18
Processing file: extracted\AA\wiki_19
Processing file: extracted\AA\wiki_20
Processing file: extracted\AA\wiki_21
Processing file: extracted\AA\wiki_22
Processing file: extracted\AA\wiki_23
Processing file: extracted\AA\wiki_24
Processing file: extracted\AA\wiki_25
Processing f

### Retrieve keywords from question

In [6]:
import spacy

nlp = spacy.load('pl_core_news_sm')
def get_keywords(question):
    doc = nlp(question)
    keywords = [token.text for token in doc if not token.is_stop]
    return " ".join(keywords)

### Searching and answer extraction

In [1]:
from transformers import pipeline
from elasticsearch import Elasticsearch

# Inicjalizacja Elasticsearch i pipeline QA
es = Elasticsearch("http://localhost:9200")
qa_pipeline = pipeline("question-answering", model="radlab/polish-qa-v2")
# qa_pipeline = pipeline("question-answering", model="henryk/bert-base-multilingual-cased-finetuned-polish-squad2")
# qa_pipeline = pipeline("question-answering", model="sdadas/polish-roberta-large-v2") <- super bad
# qa_pipeline = pipeline("question-answering", model="sdadas/polish-gpt2-xl") <- super bad as well

Device set to use cpu


In [7]:
INDEX_NAME = "wiki_index"

# 2. Funkcja pobierająca akapity z ES
def retrieve_paragraphs(question, es_client, index_name, size=10):
    """
    Wyszukuje w indeksie 'index_name' paragrafy z fields: ['title', 'content'] pasujące do 'question'.
    Zwraca listę (max 'size') tekstów - każdy to pojedynczy paragraf.
    """
    print(get_keywords(question))
    query = {
        "query": {
            "multi_match": {
                "query": question,
                "fields": ["title", "content"],
                "operator": "or",
                "minimum_should_match": "60%"
            }
        },
        "size": size
    }
    response = es_client.search(index=index_name, body=query)

    paragraphs = []
    for hit in response["hits"]["hits"]:
        # Zakładamy, że w _source mamy klucz 'content' z tekstem paragrafu
        paragraphs.append(hit["_source"]["content"])
    return paragraphs

# 3. Funkcja, która spośród pobranych paragrafów wybiera najlepszą odpowiedź
def get_best_answer(question, paragraphs):
    """
    Dla każdego paragrafu wywołuje pipeline QA. 
    Zwraca (best_answer, best_score).
    """
    best_answer = None
    best_score = float("-inf")

    for paragraph in paragraphs:
        if paragraph.strip():
            result = qa_pipeline(question=question, context=paragraph)
            if result["score"] > best_score:
                best_score = result["score"]
                best_answer = result["answer"]

    return best_answer, best_score

In [8]:
# 4. Przykładowe wywołanie

def answer_question(question, es, index_name):
    paragraphs = retrieve_paragraphs(question, es, index_name, size=10)
    best_answer, best_score = get_best_answer(question, paragraphs)
    
    print(f"Pytanie: {question}")
    print(f"Najlepsza odpowiedź: {best_answer}")
    print(f"Score: {best_score}")

In [9]:
question = "Kto stworzył Tomb Raider?"
answer_question(question, es, INDEX_NAME)

stworzył Tomb Raider
Pytanie: Kto stworzył Tomb Raider?
Najlepsza odpowiedź:  Toby Gard i Paul Howard Douglasa,
Score: 0.9921822547912598


In [10]:
question = "Jak nazywa się bohaterka gier komputerowych z serii Tomb Raider?"
answer_question(question, es, INDEX_NAME)

question = "Czy w państwach starożytnych powoływani byli posłowie i poselstwa?"
paragraphs = retrieve_paragraphs(question, es, INDEX_NAME, size=10)
question_tuned = "Czy w starożytności powoływani byli posłowie i poselstwa? Odpowiedz jednym słowem, tak lub nie."
best_answer, best_score = get_best_answer(question_tuned, paragraphs)

print(f"Pytanie: {question}")
print(f"Najlepsza odpowiedź: {best_answer}")
print(f"Score: {best_score}")

nazywa bohaterka gier komputerowych serii Tomb Raider
Pytanie: Jak nazywa się bohaterka gier komputerowych z serii Tomb Raider?
Najlepsza odpowiedź:  Lary Croft
Score: 0.9942718744277954
państwach starożytnych powoływani posłowie poselstwa
Pytanie: Czy w państwach starożytnych powoływani byli posłowie i poselstwa?
Najlepsza odpowiedź:  byli powoływani do służby wojskowej
Score: 0.10543614625930786


In [11]:
question = "Ile pełnych tygodni ma rok kalendarzowy?"
answer_question(question, es, INDEX_NAME)

pełnych tygodni rok kalendarzowy
Pytanie: Ile pełnych tygodni ma rok kalendarzowy?
Najlepsza odpowiedź:  dziesięciu
Score: 0.8816742300987244


### Evaluation

In [12]:
from difflib import SequenceMatcher
import re

def levenshtein_distance(s1, s2):
    return SequenceMatcher(None, s1, s2).ratio()

def is_textual_match(pred, gold, threshold=0.5):
    return levenshtein_distance(pred.lower(), gold.lower()) >= threshold

def is_numerical_match(pred, gold):
    pred_num = re.search(r"\d+", pred)
    gold_num = re.search(r"\d+", gold)
    if pred_num and gold_num:
        return pred_num.group() == gold_num.group()
    return False

def evaluate(in_file, expected_file):
    with open(in_file, 'r', encoding='utf-8') as file_in, open(expected_file, 'r', encoding='utf-8') as file_expected:
        questions = [line.strip() for line in file_in]
        gold_answers = [line.strip() for line in file_expected]
    
    total = len(questions)
    correct = 0

    for index, (question, gold) in enumerate(zip(questions, gold_answers)):
        if index >= 20:
            # My machine cannot handle more
            break
        paragraphs = retrieve_paragraphs(question, es, INDEX_NAME, size=10)
        best_answer, best_score = get_best_answer(question, paragraphs)
        print(f"Q: {question}\nA: {best_answer}\nExpected: {gold}\n\n")
        print(f"Score: {best_score}\n\n")
        if is_numerical_match(best_answer, gold) or is_textual_match(best_answer, gold):
            correct += 1

    accuracy = correct / total * 100
    print(f"Accuracy: {accuracy:.2%}")

DEV_0_IN = "data/dev-0/in.tsv"
DEV_0_EXPECTED = "data/dev-0/expected.tsv"
evaluate(DEV_0_IN, DEV_0_EXPECTED)

nazywa pierwsza litera alfabetu greckiego
Q: Jak nazywa się pierwsza litera alfabetu greckiego?
A: Alfa
Expected: alfa


Score: 0.953810453414917


nazywa dowolny odcinek łączący punkty okręgu
Q: Jak nazywa się dowolny odcinek łączący dwa punkty okręgu?
A:  okręgu.
Cięciwą
Expected: cięciwa


Score: 0.990360677242279


państwie rozpoczyna akcja powieści pustyni puszczy
Q: W którym państwie rozpoczyna się akcja powieści „W pustyni i w puszczy”?
A:  egipskiego
Expected: w Egipcie


Score: 0.8773928880691528


państwach starożytnych powoływani posłowie poselstwa
Q: Czy w państwach starożytnych powoływani byli posłowie i poselstwa?
A:  System taki panował głównie w państwach starożytnych
Expected: tak


Score: 0.4257482886314392


zespole występowała Hanka filmie Żona Australijczyka
Q: W jakim zespole występowała Hanka w filmie „Żona dla Australijczyka”?
A:  Meteory
Expected: Mazowsze


Score: 0.9639838933944702


państwie leży Bombaj
Q: W którym państwie leży Bombaj?
A:  indyjski
Expected